# TP Big Data - Analyse Streaming des Données TAN avec Spark

Ce notebook permet d'analyser en temps réel (streaming) les données collectées depuis l'API TAN et envoyées à Kafka. Nous effectuerons deux analyses distinctes en utilisant des fenêtres temporelles :

1. Surveillance en temps réel des arrêts de transport avec fenêtres glissantes
2. Analyse des tendances de temps d'attente par fenêtres temporelles

## Initialisation de Spark

Commençons par configurer notre session Spark pour le traitement en streaming.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, expr, to_timestamp
from pyspark.sql.functions import avg, count, min, max, sum, explode, regexp_extract
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, TimestampType
import time

# Création de la session Spark
spark = SparkSession.builder \
    .appName("TAN Streaming Analysis") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .getOrCreate()

# Réduire les messages de log
spark.sparkContext.setLogLevel("WARN")

print("Session Spark initialisée pour le streaming!")

/opt/conda/lib/python3.12/site-packages/pyspark/bin/spark-class: line 71: /usr/lib/jvm/java-17-openjdk-amd64/bin/java: No such file or directory
/opt/conda/lib/python3.12/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

## Streaming Analysis 1: Surveillance des arrêts en temps réel

Cette analyse nous permettra de surveiller les arrêts en temps réel et de calculer des statistiques sur des fenêtres de temps de 5 minutes.

### Définition du schéma et configuration du stream

In [3]:
# Définition du schéma pour les données d'arrêts
stop_schema = StructType([
    StructField("stop_code", StringType(), True),
    StructField("stop_name", StringType(), True),
    StructField("stop_distance", StringType(), True),
    StructField("ligne", ArrayType(StructType([
        StructField("numLigne", StringType(), True)
    ])), True),
    StructField("timestamp", StringType(), True)
])

# Création du stream pour la lecture depuis Kafka
stops_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092") \
    .option("subscribe", "tan_stops") \
    .option("startingOffsets", "latest") \
    .load() \
    .selectExpr("CAST(value AS STRING) AS json", "timestamp AS kafka_timestamp")

# Parsing du JSON
parsed_stops = stops_stream \
    .select(
        from_json("json", stop_schema).alias("data"),
        col("kafka_timestamp")
    ) \
    .select("data.*", "kafka_timestamp")

NameError: name 'spark' is not defined

### Traitement et analyse en streaming

In [ ]:
# Transformation du timestamp en format timestamp et extraction de la distance numérique
processed_stops = parsed_stops \
    .withColumn("event_time", to_timestamp(col("timestamp"))) \
    .withColumn("distance_meters", 
                regexp_extract("stop_distance", r"(\d+)", 1).cast(FloatType()))

# Analyse par fenêtre temporelle: statistiques sur les arrêts par intervalles de 5 minutes
stops_stats = processed_stops \
    .withWatermark("event_time", "10 minutes") \
    .groupBy(
        window(col("event_time"), "5 minutes")
    ) \
    .agg(
        count("*").alias("total_stops"),
        avg("distance_meters").alias("avg_distance"),
        min("distance_meters").alias("min_distance"),
        max("distance_meters").alias("max_distance"),
        count("stop_code").alias("num_data_points")
    ) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("total_stops"),
        col("avg_distance"),
        col("min_distance"),
        col("max_distance"),
        col("num_data_points")
    )

### Démarrage du premier streaming query

In [ ]:
# Configuration et démarrage du stream pour afficher les résultats dans la console
query1 = stops_stats \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 10) \
    .start()

print("Streaming query 1 démarrée. Analyse des arrêts par fenêtres de 5 minutes en cours...")
print("Ce streaming continuera jusqu'à ce que vous l'arrêtiez manuellement.")

Si vous souhaitez arrêter le premier stream, exécutez la cellule suivante :

In [ ]:
# Arrêt du premier stream (facultatif)
if 'query1' in locals() and query1.isActive:
    query1.stop()
    print("Premier stream arrêté.")
else:
    print("Le premier stream n'est pas actif ou n'a pas été démarré.")

## Streaming Analysis 2: Analyse des temps d'attente avec fenêtres glissantes

Cette analyse nous permettra de suivre les temps d'attente par ligne en utilisant des fenêtres glissantes de 10 minutes se déplaçant toutes les 5 minutes.

### Définition du schéma et configuration du stream

In [ ]:
# Définition du schéma pour les données de temps d'attente
wait_schema = StructType([
    StructField("ligne", StructType([
        StructField("numLigne", StringType(), True)
    ]), True),
    StructField("terminus", StringType(), True),
    StructField("arret", StructType([
        StructField("codeArret", StringType(), True),
        StructField("libelle", StringType(), True)
    ]), True),
    StructField("temps", StringType(), True),
    StructField("timestamp", StringType(), True)
])

# Création du stream pour la lecture depuis Kafka
wait_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:9092") \
    .option("subscribe", "tan_wait_times") \
    .option("startingOffsets", "latest") \
    .load() \
    .selectExpr("CAST(value AS STRING) AS json", "timestamp AS kafka_timestamp")

# Parsing du JSON
parsed_wait = wait_stream \
    .select(
        from_json("json", wait_schema).alias("data"),
        col("kafka_timestamp")
    ) \
    .select(
        col("data.ligne.numLigne").alias("line_number"),
        col("data.terminus").alias("destination"),
        col("data.arret.codeArret").alias("stop_code"),
        col("data.arret.libelle").alias("stop_name"),
        col("data.temps").alias("wait_time"),
        col("data.timestamp"),
        col("kafka_timestamp")
    )

### Traitement et analyse en streaming avec fenêtres glissantes

In [ ]:
# Extraction des minutes d'attente et conversion du timestamp
processed_wait = parsed_wait \
    .withColumn("event_time", to_timestamp(col("timestamp"))) \
    .withColumn("wait_minutes", 
                regexp_extract("wait_time", r"(\d+)", 1).cast(FloatType()))

# Analyse par fenêtre glissante: temps d'attente moyen par ligne sur des fenêtres de 10 minutes glissant toutes les 5 minutes
wait_stats_by_line = processed_wait \
    .withWatermark("event_time", "15 minutes") \
    .groupBy(
        window(col("event_time"), "10 minutes", "5 minutes"),
        col("line_number")
    ) \
    .agg(
        avg("wait_minutes").alias("avg_wait"),
        count("*").alias("count"),
        min("wait_minutes").alias("min_wait"),
        max("wait_minutes").alias("max_wait")
    ) \
    .filter(col("count") > 1) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("line_number"),
        col("avg_wait"),
        col("min_wait"),
        col("max_wait"),
        col("count")
    )

### Démarrage du deuxième streaming query

In [ ]:
# Configuration et démarrage du stream pour afficher les résultats dans la console
query2 = wait_stats_by_line \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 15) \
    .start()

print("Streaming query 2 démarrée. Analyse des temps d'attente par fenêtres glissantes en cours...")
print("Ce streaming continuera jusqu'à ce que vous l'arrêtiez manuellement.")

Si vous souhaitez arrêter le deuxième stream, exécutez la cellule suivante :

In [ ]:
# Arrêt du deuxième stream (facultatif)
if 'query2' in locals() and query2.isActive:
    query2.stop()
    print("Deuxième stream arrêté.")
else:
    print("Le deuxième stream n'est pas actif ou n'a pas été démarré.")

## Bonus: Analyse des tendances par arrêt avec fenêtres temporelles

Pour une analyse plus détaillée, nous pouvons également suivre les temps d'attente par arrêt.

In [ ]:
# Analyse par fenêtre temporelle: temps d'attente moyen par arrêt sur des fenêtres de 15 minutes
wait_stats_by_stop = processed_wait \
    .withWatermark("event_time", "20 minutes") \
    .groupBy(
        window(col("event_time"), "15 minutes"),
        col("stop_name")
    ) \
    .agg(
        avg("wait_minutes").alias("avg_wait"),
        count("*").alias("count")
    ) \
    .filter(col("count") > 1) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("stop_name"),
        col("avg_wait"),
        col("count")
    )

In [ ]:
# Configuration et démarrage du stream pour afficher les résultats dans la console
query3 = wait_stats_by_stop \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 15) \
    .start()

print("Streaming query 3 démarrée. Analyse des temps d'attente par arrêt en cours...")
print("Ce streaming continuera jusqu'à ce que vous l'arrêtiez manuellement.")

Si vous souhaitez arrêter le troisième stream, exécutez la cellule suivante :

In [ ]:
# Arrêt du troisième stream (facultatif)
if 'query3' in locals() and query3.isActive:
    query3.stop()
    print("Troisième stream arrêté.")
else:
    print("Le troisième stream n'est pas actif ou n'a pas été démarré.")

## Analyse avancée: Détection des anomalies de temps d'attente

Nous allons maintenant mettre en place une détection d'anomalies simple pour identifier les lignes ayant des temps d'attente inhabituellement longs par rapport à la moyenne.

In [ ]:
# Analyse par fenêtre glissante avec détection d'anomalies
# On considère un temps d'attente comme anormal s'il dépasse 1.5 fois la moyenne calculée sur les dernières fenêtres
wait_anomalies = processed_wait \
    .withWatermark("event_time", "20 minutes") \
    .groupBy(
        window(col("event_time"), "10 minutes", "5 minutes"),
        col("line_number"),
        col("stop_name")
    ) \
    .agg(
        avg("wait_minutes").alias("avg_wait"),
        max("wait_minutes").alias("max_wait"),
        count("*").alias("count")
    ) \
    .filter(col("count") > 1) \
    .filter(col("max_wait") > 15) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("line_number"),
        col("stop_name"),
        col("avg_wait"),
        col("max_wait"),
        col("count")
    )

In [ ]:
# Configuration et démarrage du stream pour afficher les anomalies dans la console
query4 = wait_anomalies \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 10) \
    .start()

print("Streaming query 4 démarrée. Détection d'anomalies en cours...")
print("Ce streaming continuera jusqu'à ce que vous l'arrêtiez manuellement.")

Si vous souhaitez arrêter le quatrième stream, exécutez la cellule suivante :

In [ ]:
# Arrêt du quatrième stream (facultatif)
if 'query4' in locals() and query4.isActive:
    query4.stop()
    print("Quatrième stream arrêté.")
else:
    print("Le quatrième stream n'est pas actif ou n'a pas été démarré.")

## Arrêt de tous les streams

Pour arrêter tous les streams en cours, exécutez cette cellule.

In [ ]:
# Arrêt de tous les streams
for query_name in ['query1', 'query2', 'query3', 'query4']:
    if query_name in locals() and eval(f"{query_name}.isActive"):
        eval(f"{query_name}.stop()")
        print(f"{query_name} arrêté.")

print("Tous les streams sont maintenant arrêtés.")

## Conclusion

Dans ce notebook, nous avons mis en place plusieurs analyses en streaming sur les données de transport en commun de Nantes :

1. **Surveillance des arrêts en temps réel** : Nous avons analysé les données d'arrêts sur des fenêtres temporelles de 5 minutes, ce qui nous permet de suivre l'évolution de la distribution des arrêts et des distances en temps réel.

2. **Analyse des temps d'attente** : Nous avons utilisé des fenêtres glissantes de 10 minutes (se déplaçant toutes les 5 minutes) pour analyser l'évolution des temps d'attente par ligne, ce qui permet de détecter des tendances ou des problèmes de ponctualité.

3. **Analyse par arrêt** : Nous avons également suivi les temps d'attente par arrêt, ce qui peut aider à identifier les stations ayant des problèmes récurrents de service.

4. **Détection d'anomalies** : Enfin, nous avons mis en place une détection simple d'anomalies pour identifier les situations où les temps d'attente sont anormalement longs, ce qui pourrait indiquer des perturbations ou des incidents.

Ces analyses en streaming fournissent une vue en temps réel du réseau de transport, ce qui est crucial pour la supervision opérationnelle et l'amélioration continue du service.